In [ ]:
#https://www.teamrankings.com/nfl/stat/points-per-game

In [28]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

team_list = ["Arizona", "Atlanta", "Baltimore", "Buffalo", "Carolina", "Chicago", "Cincinnati", "Cleveland", 
            "Dallas", "Denver", "Detroit", "Green Bay", "Houston", "Indianapolis", "Jacksonville", "Kansas City", 
            "LA Chargers", "LA Rams", "Las Vegas", "Miami", "Minnesota", "New England", "New Orleans", "NY Giants", 
            "NY Jets", "Philadelphia", "Pittsburgh", "San Francisco", "Seattle", "Tampa Bay", "Tennessee", "Washington"]

# Note that 2023-01-01 is the 2022 regular season ish (I just estimated based on right before the playoffs typically are)
year_list = ["2023-01-01", "2022-01-01", "2021-01-01", "2020-01-01", "2019-01-01", "2018-01-01", "2017-01-01", "2016-01-01", 
            "2015-01-01", "2014-01-01", "2013-01-01", "2012-01-01", "2011-01-01", "2010-01-01", "2009-01-01", "2008-01-01", 
            "2007-01-01", "2006-01-01", "2005-01-01", "2004-01-01"]

link_list = ["https://www.teamrankings.com/nfl/stat/points-per-game", 
            "https://www.teamrankings.com/nfl/stat/red-zone-scoring-pct", 
            "https://www.teamrankings.com/nfl/stat/yards-per-game", 
            "https://www.teamrankings.com/nfl/stat/yards-per-play", 
            "https://www.teamrankings.com/nfl/stat/average-time-of-possession-net-of-ot", 
            "https://www.teamrankings.com/nfl/stat/third-down-conversion-pct", 
            "https://www.teamrankings.com/nfl/stat/yards-per-rush-attempt", 
            "https://www.teamrankings.com/nfl/stat/rushing-attempts-per-game", 
            "https://www.teamrankings.com/nfl/stat/pass-attempts-per-game", 
            "https://www.teamrankings.com/nfl/stat/completions-per-game", 
            "https://www.teamrankings.com/nfl/stat/yards-per-completion", 
            "https://www.teamrankings.com/nfl/stat/average-team-passer-rating", 
            "https://www.teamrankings.com/nfl/stat/kicking-points-per-game", 
            "https://www.teamrankings.com/nfl/stat/net-punt-yards-per-game", 
            "https://www.teamrankings.com/nfl/stat/field-goal-conversion-pct", 
            "https://www.teamrankings.com/nfl/stat/opponent-points-per-game", 
            "https://www.teamrankings.com/nfl/stat/opponent-red-zone-scoring-pct", 
            "https://www.teamrankings.com/nfl/stat/opponent-yards-per-game", 
            "https://www.teamrankings.com/nfl/stat/opponent-yards-per-play", 
            "https://www.teamrankings.com/nfl/stat/opponent-average-time-of-possession-net-of-ot", 
            "https://www.teamrankings.com/nfl/stat/opponent-third-down-conversion-pct", 
            "https://www.teamrankings.com/nfl/stat/opponent-yards-per-rush-attempt", 
            "https://www.teamrankings.com/nfl/stat/opponent-rushing-attempts-per-game", 
            "https://www.teamrankings.com/nfl/stat/opponent-pass-attempts-per-game", 
            "https://www.teamrankings.com/nfl/stat/opponent-completion-pct", 
            "https://www.teamrankings.com/nfl/stat/opponent-yards-per-completion", 
            "https://www.teamrankings.com/nfl/stat/opponent-average-team-passer-rating", 
            "https://www.teamrankings.com/nfl/stat/opponent-kicking-points-per-game", 
            "https://www.teamrankings.com/nfl/stat/opponent-net-punt-yards-per-game", 
            "https://www.teamrankings.com/nfl/stat/opponent-field-goal-conversion-pct"]


#https://www.teamrankings.com/nfl/stat/points-per-game?date=2023-01-01

In [ ]:
dataframe = pd.DataFrame()
first_column = True
for link in link_list:
    first_year = True
    for year in year_list:
        #Blank data:
        stat_name = link[link.index('stat/')+5:]
        data_dictionary = {"Team":[], stat_name:[]}
        
        #Concat the link and year in a URL
        url = link + "?date=" + year
        contents = requests.get(url, headers=headers).text
        html = BeautifulSoup(contents, 'html.parser')
        
        for team in team_list:
            value = html.find("td", {"data-sort":team}).find_next_sibling("td").text
            
            team_name = team+str(int(year[0:4])-1)
            data_dictionary['Team'].append(team_name)
            data_dictionary[stat_name].append(value)
          
        new_dataframe = pd.DataFrame(data_dictionary)
        if len(dataframe) == 0:
            dataframe = new_dataframe
        elif first_column == True:
            dataframe = pd.concat([dataframe, new_dataframe])
        elif first_year == True:
            new_dataframe.set_index('Team', inplace = True)
            dataframe = dataframe.merge(new_dataframe, left_index = True, right_index = True, how='left')
        else:
            new_dataframe.set_index('Team', inplace = True)
            dataframe.update(new_dataframe, overwrite = False)
        first_year = False
        
    if first_column == True:
        dataframe.set_index('Team', inplace = True)
        first_column = False #Now we have completed one column
        
dataframe.head(10)

In [ ]:
dataframe.to_csv("NFL Data.csv", index=True)